In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense  # Replace LSTM(64) with GRU(64) for lighter computing
from sklearn.metrics import classification_report

In [2]:
# Load IMDB dataset with the top 10,000 most frequent words
vocab_size = 10000
max_length = 200  # max words per review

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 120s 367ms/step - accuracy: 0.6840 - loss: 0.5722 - val_accuracy: 0.8472 - val_loss: 0.3546
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 108s 345ms/step - accuracy: 0.8633 - loss: 0.3321 - val_accuracy: 0.8386 - val_loss: 0.3860
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 144s 351ms/step - accuracy: 0.8836 - loss: 0.2861 - val_accuracy: 0.8490 - val_loss: 0.3739


In [5]:
y_pred_prob = model.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

print(classification_report(y_test, y_pred))

782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step
              precision    recall  f1-score   support

           0       0.91      0.77      0.83     12500
           1       0.80      0.92      0.86     12500

    accuracy                           0.85     25000
   macro avg       0.86      0.85      0.85     25000
weighted avg       0.86      0.85      0.85     25000



Thanks for uploading **`DL3 Time Series LSTM.ipynb`**. This notebook implements an **LSTM (Long Short-Term Memory)** model using PyTorch to learn patterns in time series data, likely for forecasting or sequence prediction.

Here’s a **line-by-line breakdown**, explaining how and why each part works.

---

## ✅ LSTM Time Series Forecasting: Line-by-Line Explanation

---

### 📦 Imports

```python
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
```

* `torch`, `nn`: For model creation and operations.
* `optim`: Optimization algorithms (e.g., Adam).
* `numpy`: For array manipulation.
* `matplotlib.pyplot`: Plotting the input and predictions.

---

### 🔧 Generate Sine Wave Dataset

```python
x = np.linspace(0, 100, 1000)
y = np.sin(x)
```

* `x`: Evenly spaced values from 0 to 100.
* `y`: Sine wave values → the time series to learn.

---

### 🪄 Create Sequence Dataset

```python
def create_dataset(series, seq_length):
    X, Y = [], []
    for i in range(len(series) - seq_length):
        X.append(series[i:i + seq_length])
        Y.append(series[i + seq_length])
    return np.array(X), np.array(Y)
```

* For each point in the time series, take:

  * A sequence of `seq_length` values as input `X`.
  * The next value as the target `Y`.

---

### 🔁 Prepare Training Data

```python
seq_length = 50
X, Y = create_dataset(y, seq_length)
X = torch.FloatTensor(X).unsqueeze(-1)  # shape: [samples, seq_len, 1]
Y = torch.FloatTensor(Y).unsqueeze(-1)  # shape: [samples, 1]
```

* Converts NumPy arrays to PyTorch tensors.
* Adds last dimension to represent a single feature (`[batch, seq, feature]`).

---

### 🧠 Define LSTM Model

```python
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)  # out: [batch, seq, hidden]
        out = self.fc(out[:, -1, :])  # Take the last timestep's output
        return out
```

* `nn.LSTM`: Learns sequential dependencies.
* `batch_first=True`: Ensures shape `[batch, seq, feature]`.
* Final FC layer maps last hidden state to a single value (prediction).
* Only last timestep’s output is used for forecasting the next value.

---

### ⚙️ Model, Optimizer, Loss

```python
model = LSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
```

* `MSELoss`: Measures how close prediction is to target.
* `Adam`: Adaptive learning rate for faster convergence.

---

### 🔁 Training Loop

```python
epochs = 100
losses = []

for epoch in range(epochs):
    model.train()
    output = model(X)
    loss = criterion(output, Y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
```

* Forward pass: model predicts next value.
* Loss computed and backpropagated.
* Optimizer updates weights.
* Logs loss every 10 epochs.

---

### 📉 Plot Training Loss

```python
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.grid(True)
plt.show()
```

* Shows how well the model learns to minimize error.

---

### 🔮 Predict & Plot Future Values

```python
model.eval()
preds = []

input_seq = X[0].unsqueeze(0)  # start with the first window

for _ in range(100):
    with torch.no_grad():
        pred = model(input_seq)
        preds.append(pred.item())

        new_input = torch.cat((input_seq[:, 1:, :], pred.unsqueeze(0).unsqueeze(2)), dim=1)
        input_seq = new_input
```

* **Autoregressive prediction**:

  * Predict 1 value → append to input → predict next → repeat.
  * Only initial 50 points are from true data; rest are predicted by the model.

---

### 📈 Plot Predictions vs. Ground Truth

```python
plt.plot(y[seq_length:seq_length+100], label='True')
plt.plot(preds, label='Predicted')
plt.legend()
plt.title("Sine Wave Forecasting")
plt.grid(True)
plt.show()
```

* Compares real vs predicted future points.
* Shows how well the model generalizes to unseen time steps.

---

## ✅ Summary

| Component        | Role                                                    |
| ---------------- | ------------------------------------------------------- |
| `create_dataset` | Converts time series into supervised learning format.   |
| `LSTM` model     | Learns time-dependent features.                         |
| `MSELoss`        | Regression loss to minimize forecast error.             |
| `input_seq` loop | Predicts future points one at a time (auto-regressive). |

---

Would you like:

* A **visual of the LSTM architecture**, or
* An extension to make **multi-step forecasting** or **multivariate time series**?
